<a href="https://colab.research.google.com/github/Riky2014/NAPDE/blob/main/Project1/Cp2/Checkpoint2_BFGS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import required libraries

import tensorflow as tf
import numpy as np
import scipy.io
from tensorflow import keras
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.model_selection import train_test_split
!pip -q install pyDOE
from pyDOE import lhs  # for latin hypercube sampling

In [3]:
!git clone https://github.com/Alepescinaa/ScientificTools
%cd ScientificTools/Project1/Cp2

CP2data = np.load("CP2data.npz")
CP2data = CP2data['arr_0']

fatal: destination path 'ScientificTools' already exists and is not an empty directory.
/content/ScientificTools/Project1/Cp2


In [4]:

"""
    Optimize a keras model using scipy.optimize
    This block of code is taken and adapted from https://github.com/pedro-r-marques/keras-opt/tree/master
    See the repository for all the information.
"""
import numpy as np
from scipy.optimize import minimize
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import backend as K  # pylint: disable=import-error

from tensorflow.python.keras.engine import data_adapter


class ScipyOptimizer():
    """ Implements a training function that uses scipy optimize in order
        to determine the weights for the model.

        The minimize function expects to be able to attempt multiple solutions
        over the model. It calls a function which collects all gradients for
        all steps and then returns the gradient information to the optimizer.
    """

    def __init__(self, model, method='BFGS', verbose=1, maxiter=1):
        self.model = model
        self.method = method
        self.verbose = verbose
        self.maxiter = maxiter
        if model.run_eagerly:
            self.func = model.__call__
        else:
            self.func = tf.function(
                model.__call__, experimental_relax_shapes=True)

    def _update_weights(self, x):
        x_offset = 0
        for var in self.model.trainable_variables:
            shape = var.get_shape()
            w_size = np.prod(shape)
            value = np.array(x[x_offset:x_offset+w_size]).reshape(shape)
            K.set_value(var, value)
            x_offset += w_size
        assert x_offset == len(x)

    def _fun_generator(self, x, iterator):
        """ Function optimized by scipy minimize.

            Returns function cost and gradients for all trainable variables.
        """
        model = self.model
        self._update_weights(x)
        losses = []

        dataset = iterator._dataset  # pylint:disable=protected-access
        assert dataset is not None
        iterator = iter(dataset)

        size = dataset.cardinality().numpy()
        if size > 0:
            n_steps = (size + dataset.batch_size - 1) // dataset.batch_size
        else:
            n_steps = None

        progbar = keras.utils.Progbar(n_steps, verbose=self.verbose)

        with tf.GradientTape() as tape:
            for step, data in enumerate(iterator):
                data = data_adapter.expand_1d(data)
                x, y, sample_weight = data_adapter.unpack_x_y_sample_weight(
                    data)
                y_pred = self.func(x, training=True)
                loss = model.compiled_loss(y, y_pred, sample_weight,
                                           regularization_losses=model.losses)
                progbar.update(step, [('loss', loss.numpy())])
                losses.append(loss)
            xloss = tf.reduce_mean(tf.stack(losses))
            grads = tape.gradient(xloss, model.trainable_variables)

        cost = xloss.numpy()

        if all(isinstance(x, tf.Tensor) for x in grads):
            xgrads = np.concatenate([x.numpy().reshape(-1) for x in grads])
            return cost, xgrads

        if all(isinstance(x, tf.IndexedSlices) for x in grads):
            xgrad_list = []
            for var, grad in zip(model.trainable_variables, grads):
                value = tf.Variable(np.zeros(var.shape), dtype=var.dtype)
                value.assign_add(grad)
                xgrad_list.append(value.numpy())
            xgrads = np.concatenate([x.reshape(-1) for x in xgrad_list])
            return cost, xgrads

        raise NotImplementedError()
        return -1, np.array([])  # pylint:disable=unreachable

    def train_function(self, iterator):
        """ Called by model fit.
        """
        min_options = {
            'maxiter': self.maxiter,
            'disp': bool(self.verbose),
        }

        var_list = self.model.trainable_variables
        x0 = np.concatenate([x.numpy().reshape(-1) for x in var_list])

        result = minimize(
            self._fun_generator, x0, method=self.method, jac=True,
            options=min_options, args=(iterator,))

        self._update_weights(result['x'])
        return {'loss': result['fun']}


def make_train_function(model, **kwargs):
    """ Returns a function that will be called to train the model.

        model._steps_per_execution must be set in order for train function to
        be called once per epoch.
    """
    model._assert_compile_was_called()  # pylint:disable=protected-access
    model._configure_steps_per_execution(tf.int64.max)  # pylint:disable=protected-access
    opt = ScipyOptimizer(model, **kwargs)
    return opt.train_function

In [5]:
# set seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [6]:
# collocation points
Ncl = 10000
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

In [7]:
def penalty(param, lower_bound, upper_bound):
    return tf.reduce_sum(tf.square(tf.maximum(param - upper_bound, 0)) +
                         tf.square(tf.maximum(lower_bound - param, 0)))
# Residual loss
@tf.function
def r_PINN(x,y,param1, param2, param3):
    input_data=tf.concat([x,y],1)
    u = PINN_base(input_data)
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    pi = tf.constant(np.pi,dtype=tf.float64)
    param1
    theta0 = pi/2 - param1
    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    D_00 = 1 / param2 * a[0]**2 + b[0]**2
    D_01 = 1 / param2 * a[0] * a[1] + b[0] * b[1]
    D_10 = 1 / param2 * a[0] * a[1] + b[0] * b[1]
    D_11 = 1 / param2 * a[1]**2 + b[1]**2

    return ((u_x * D_00 * u_x + u_x * D_01 * u_y + u_y * D_10 * u_x + u_y * D_11 * u_y))  - 1/100**2

# PINN loss function
def my_loss_fn(y_true, y_pred, xcl = xcl, ycl = ycl):

    param1 = tf.reduce_mean(y_pred[:,1])
    param2 = tf.reduce_mean(y_pred[:,2])
    param3 = tf.reduce_mean(y_pred[:,3])
    r_pred = r_PINN(xcl,ycl,param1, param2, param3)
    """
    tf.print(param1)
    tf.print(param2)
    tf.print(param3)
    """
    # loss components
    mse_meas = tf.reduce_mean(tf.pow( y_true[:,0]-y_pred[:,0], 2 ))
    mse_r  = tf.reduce_mean(tf.abs(r_pred))

    # bc
    p1_tensor = tf.expand_dims(tf.convert_to_tensor(param1, dtype=tf.float64), axis=0)
    p2_tensor = tf.expand_dims(tf.convert_to_tensor(param2, dtype=tf.float64), axis=0)
    p3_tensor = tf.expand_dims(tf.convert_to_tensor(param3, dtype=tf.float64), axis=0)
    origin = tf.expand_dims(tf.convert_to_tensor([1.5, param3], dtype=tf.float64), axis=0)

    mse_bc = tf.pow(PINN([origin, p1_tensor, p2_tensor, p3_tensor])[:,0], 2)

    #penalty over param boundaries
    mse_penalty = penalty(param1,-np.pi/10,np.pi/10)+penalty(param2,1,9)+penalty(param3,-1.5,1.5)

    return mse_meas + mse_r + mse_bc + mse_penalty

In [8]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import LSTM

regularization_strength = 1e-3

PINN_base = tf.keras.Sequential([
    tf.keras.layers.Dense(32, activation='relu', input_shape=(2,),
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),

    #tf.keras.layers.Reshape((1, 32)),

    #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units=64)),

    tf.keras.layers.Dense(128, activation='relu',
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),

    tf.keras.layers.Dense(64, activation='relu',
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64),

    tf.keras.layers.Dense(1, activation=None,
                          kernel_initializer="glorot_uniform",
                          kernel_regularizer=regularizers.l2(regularization_strength),
                          dtype=tf.float64)
])

In [9]:
from scipy.interpolate import RBFInterpolator

def checkpoint1_solution(x, y, t, X, Y, s_value=0.05, s_aniso_1=0.5, s_aniso_2=0.5):
    coordinates = np.column_stack((x, y))

    mesh_coordinates=np.column_stack((X.ravel(), Y.ravel()))

    s = [s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_value,s_aniso_1, s_value,s_value,s_value,s_value, s_aniso_2,s_value,s_value,s_value,s_value]

    rbf = RBFInterpolator(coordinates, t, neighbors=None, smoothing=s, kernel='thin_plate_spline', epsilon=None, degree=1)

    time_pred = rbf(mesh_coordinates)
    time_pred=time_pred.reshape(1501,1501)

    return time_pred

In [10]:
ind_disp = 0
x = CP2data[ind_disp][0]
y = CP2data[ind_disp][1]
t = CP2data[ind_disp][2]

In [11]:
def custom_bias_initializer_theta(shape, dtype=None):
    return tf.constant(0.01, shape = shape, dtype=dtype)

def custom_bias_initializer_a_aniso(shape, dtype=None):
    return tf.constant(4, shape = shape, dtype=dtype)

def custom_bias_initializer_y0(shape, dtype=None):
    return tf.constant(y0_initial, shape = shape, dtype=dtype)

In [12]:
#train/val split
xmeas_train, xmeas_val, ymeas_train, ymeas_val, tmeas_train, tmeas_val = train_test_split(x, y, t, test_size=0.1)
xmeas_train = tf.constant(xmeas_train.reshape(18, 1), dtype=tf.float64)
ymeas_train = tf.constant(ymeas_train.reshape(18, 1), dtype=tf.float64)
tmeas_train = tf.constant(tmeas_train.reshape(18, 1), dtype=tf.float64)
xmeas_val = tf.constant(xmeas_val.reshape(2, 1), dtype=tf.float64)
ymeas_val = tf.constant(ymeas_val.reshape(2, 1), dtype=tf.float64)
tmeas_val = tf.constant(tmeas_val.reshape(2, 1), dtype=tf.float64)

# y0 initial guess
X, Y = np.meshgrid(np.linspace(-1.5,1.5,1501), np.linspace(-1.5,1.5,1501))
time_pred = checkpoint1_solution(x, y, t, X, Y, s_value=0.05, s_aniso_1=0.5, s_aniso_2=0.5)
y0_initial = Y[np.where(time_pred==np.min(time_pred))]

# parameter is treated as the bias of an additional layer that always receives a zero input.

inputs = tf.keras.Input(shape=(2,))
input1 = tf.keras.Input(shape=(1,), dtype=tf.float64)
input2 = tf.keras.Input(shape=(1,), dtype=tf.float64)
input3 = tf.keras.Input(shape=(1,), dtype=tf.float64)
emb1 = tf.keras.layers.Dense(1, input_shape=(1,), bias_initializer = custom_bias_initializer_theta, dtype=tf.float64)
emb2 = tf.keras.layers.Dense(1, input_shape=(1,), bias_initializer = custom_bias_initializer_a_aniso, dtype=tf.float64)
emb3 = tf.keras.layers.Dense(1, input_shape=(1,), bias_initializer = custom_bias_initializer_y0, dtype=tf.float64)
output = tf.concat( [PINN_base(inputs), emb1(input1), emb2(input2), emb3(input3) ] , 1 )

PINN = tf.keras.Model(inputs=[inputs,input1,input2,input3], outputs=output)

PINN.compile(loss = my_loss_fn, optimizer=tf.keras.optimizers.Adam(learning_rate=0.002,beta_1=0.99))

history_adam = PINN.fit( [ tf.concat([xmeas_train,ymeas_train],1), tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train)) ], tmeas_train , epochs=120)

PINN.compile(loss = my_loss_fn)

PINN.train_function = make_train_function(PINN, maxiter=5)

history = PINN.fit( [ tf.concat([xmeas_train,ymeas_train],1) , tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train)) ], tmeas_train)

param = tf.reduce_mean(PINN([tf.concat([xmeas_train,ymeas_train],1) , tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train))])[:,1])
print(param)
"""
# Adam optimizer
initial_learning_rate = 0.002
tf_optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate,beta_1=0.99)

patience = float('inf')
patience_lr = 500
min_delta = 1e-9
best_val_loss = float('inf')
wait = 0
count = 0

for iter in range(12000):

  # compute gradients using AD
  loss_value,grads,grad_param = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, param)

  # update neural network weights
  tf_optimizer.apply_gradients(zip(grads+[grad_param],PINN.trainable_variables+[param]))

  loss_value_val, _, _ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val, param)

  best_weigths = None
  best_params = None

  # Early stopping
  if loss_value_val < best_val_loss - min_delta:
      best_val_loss = loss_value_val
      wait = 0
      count = 0
      best_weights = PINN.get_weights()
      best_params = param.numpy()
  else:
      wait += 1
      count += 1

      if count >= patience_lr:
        tf_optimizer.learning_rate = tf_optimizer.learning_rate * 0.9
        count = 0

      if wait >= patience:
          print('Early stopping at epoch', iter + 1)
          break

  # display intermediate results
  if ((iter+1) % 100 == 0):
    print('iter =  '+str(iter+1))
    #loss_value_np=loss_value.numpy()
    #print('loss = {:.4f}'.format(loss_value_np))
    tf.print('loss =' , loss_value)
    tf.print('loss_val_param =' , loss_value_val)

    print(param.numpy())
"""


Epoch 1/120
1/1 [==============================] - 12s 12s/step - loss: 0.1797
Epoch 2/120
1/1 [==============================] - 0s 55ms/step - loss: 0.1440
Epoch 3/120
1/1 [==============================] - 0s 50ms/step - loss: 0.1515
Epoch 4/120
1/1 [==============================] - 0s 57ms/step - loss: 0.1523
Epoch 5/120
1/1 [==============================] - 0s 65ms/step - loss: 0.1429
Epoch 6/120
1/1 [==============================] - 0s 59ms/step - loss: 0.1337
Epoch 7/120
1/1 [==============================] - 0s 41ms/step - loss: 0.1288
Epoch 8/120
1/1 [==============================] - 0s 42ms/step - loss: 0.1285
Epoch 9/120
1/1 [==============================] - 0s 49ms/step - loss: 0.1295
Epoch 10/120
1/1 [==============================] - 0s 37ms/step - loss: 0.1292
Epoch 11/120
1/1 [==============================] - 0s 37ms/step - loss: 0.1262
Epoch 12/120
1/1 [==============================] - 0s 41ms/step - loss: 0.1210
Epoch 13/120
1/1 [==============================]

/usr/local/lib/python3.10/dist-packages/scipy/optimize/_minimize.py:705: OptimizeWarning: Maximum number of iterations has been exceeded.
  res = _minimize_bfgs(fun, x0, args, jac, callback, **options)


"\n# Adam optimizer\ninitial_learning_rate = 0.002\ntf_optimizer = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate,beta_1=0.99)\n\npatience = float('inf')\npatience_lr = 500\nmin_delta = 1e-9\nbest_val_loss = float('inf')\nwait = 0\ncount = 0\n\nfor iter in range(12000):\n\n  # compute gradients using AD\n  loss_value,grads,grad_param = grad(PINN,xcl,ycl,xmeas_train, ymeas_train, tmeas_train, param)\n\n  # update neural network weights\n  tf_optimizer.apply_gradients(zip(grads+[grad_param],PINN.trainable_variables+[param]))\n\n  loss_value_val, _, _ = grad(PINN, xcl, ycl, xmeas_val, ymeas_val, tmeas_val, param)\n\n  best_weigths = None\n  best_params = None\n\n  # Early stopping\n  if loss_value_val < best_val_loss - min_delta:\n      best_val_loss = loss_value_val\n      wait = 0\n      count = 0\n      best_weights = PINN.get_weights()\n      best_params = param.numpy()\n  else:\n      wait += 1\n      count += 1\n\n      if count >= patience_lr:\n        tf_optimizer.le

In [16]:
param1 = tf.reduce_mean(PINN([tf.concat([xmeas_train,ymeas_train],1) , tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train))])[:,1])
param2 = tf.reduce_mean(PINN([tf.concat([xmeas_train,ymeas_train],1) , tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train))])[:,2])
param3 = tf.reduce_mean(PINN([tf.concat([xmeas_train,ymeas_train],1) , tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train)), tf.zeros(tf.shape(xmeas_train))])[:,3])
print(param1, param2, param3)

tf.Tensor(0.14161838248267763, shape=(), dtype=float64) tf.Tensor(4.144833637166823, shape=(), dtype=float64) tf.Tensor(0.5775586228917283, shape=(), dtype=float64)
